In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

Connect the DB

Constants

In [44]:
# UUIDs
uuid_hp = '34cfe992-c2c0-11ea-9026-02e7594ce0a0'
# connection parameters
dbname = "eamena"
user = "eamenar"
password = "eamenar"
host = "52.50.27.140"
port = "5432"

In [41]:
try:
    connection = pg.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cur = connection.cursor()
    print("Connection established successfully!")
except pg.Error as e:
    print(f"Error: {e}")

Connection established successfully!


Select an HP

In [37]:
selected_hp = 'EAMENA-0500002'

Get the UUID of the HP from its ID

In [47]:
sqll = """
SELECT
      resourceinstanceid AS resourceid
      FROM tiles
      WHERE tiledata -> '%s' -> 'en' ->> 'value' LIKE '%s'
""" % (uuid_hp, selected_hp)
# print(sqll)
cur.execute(sqll)
hpid = cur.fetchone()[0]   
print("the UUID of '" + selected_hp + "' is '" + hpid + "'")

the UUID of 'EAMENA-0500002' is 'dbc95d2d-38fb-465e-a6cb-0545eaa7584f'


Use the HP UUID (ex: `EAMENA-0500002`) to collect data. Field list are:

* Condition: `34cfe9f5-c2c0-11ea-9026-02e7594ce0a0`
* Damage Extent Type: `34cfea8e-c2c0-11ea-9026-02e7594ce0a0`
* ...

In [46]:
sqll = """
SELECT value FROM values 
WHERE valueid::text IN
(
SELECT tiledata ->> '%s' AS DamageExtentType
FROM tiles 
WHERE resourceinstanceid::text LIKE '%s'
AND tiledata -> '%s' IS NOT NULL
)
""" % (uuid_hp, hpid, uuid_hp)
print(sqll)
cur.execute(sqll)
outme = cur.fetchall()
print(outme)
  


SELECT value FROM values 
WHERE valueid::text IN
(
SELECT tiledata ->> '34cfe992-c2c0-11ea-9026-02e7594ce0a0' AS DamageExtentType
FROM tiles 
WHERE resourceinstanceid::text LIKE 'dbc95d2d-38fb-465e-a6cb-0545eaa7584f'
AND tiledata -> '34cfe992-c2c0-11ea-9026-02e7594ce0a0' IS NOT NULL
)

[]


Read the TSV file

In [52]:
tsv_file = "https://raw.githubusercontent.com/eamena-project/eamena-arches-dev/main/dev/data_quality/output.tsv"
df = pd.read_csv(tsv_file, delimiter='\t')
df = df[["level1", "level2", "level3", "uuid_sql"]]
df = df.dropna()
print(df.to_markdown())

|    | level1             | level2              | level3                   | uuid_sql                             |
|---:|:-------------------|:--------------------|:-------------------------|:-------------------------------------|
|  2 | ASSESSMENT SUMMARY | ASSESSMENT ACTIVITY | Assessment Activity Type | 34cfea4d-c2c0-11ea-9026-02e7594ce0a0 |


In [51]:
level1_values
variable = level1_values.tolist()
variable
listed_uuids = df['uuid_sql']
listed_uuids
# unique values level 1
level1_values = df['level1'].unique()

0                                      NaN
1                                      NaN
2     34cfea4d-c2c0-11ea-9026-02e7594ce0a0
3                                      NaN
4                                      NaN
                      ...                 
93                                     NaN
94                                     NaN
95                                     NaN
96                                     NaN
97                                     NaN
Name: uuid_sql, Length: 98, dtype: object

In [ ]:
value = [8, 12, 7, 14, 10]
variable = level1_values.tolist()

# Sample data
df = pd.DataFrame(dict(
    value = [8, 12, 7, 14, 10],
    variable = ['V1', 'V2', 'V3', 'V4', 'V5']))
           
fig = px.line_polar(df, r = 'value', theta = 'variable', line_close = True)

fig.show()
